In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Load the preprocessed data: Load the preprocessed data from the csv files using the read_csv() function.

In [ ]:
train_data = pd.read_csv('train_data.csv', index_col=0)
test_data = pd.read_csv('test_data.csv', index_col=0)


Scale the data: Scale the data using the MinMaxScaler from the sklearn library. This will ensure that all the features are in the same range.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# create the scaler object
scaler = MinMaxScaler()

# fit the scaler on the training data and transform it
train_data_scaled = scaler.fit_transform(train_data)

# transform the testing data using the same scaler
test_data_scaled = scaler.transform(test_data)


Convert data to 3D format: Convert the data into a 3D format of [samples, time steps, features]. In this case, we will use a time step of 60 days, which means we will use the stock prices of the previous 60 days to predict the stock price of the next day.

In [ ]:
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i+time_step])
        y.append(data[i+time_step])
    return np.array(X), np.array(y)

time_step = 60
X_train, y_train = create_dataset(train_data_scaled, time_step)
X_test, y_test = create_dataset(test_data_scaled, time_step)

Reshape the data: Reshape the data into a 3D format of [samples, time steps, features].

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape) 

In [ ]:
# Select the first feature column from X_train and X_test
X_train_selected = X_train[:, :, 0]
X_test_selected = X_test[:, :, 0]

# Reshape X_train_selected and X_test_selected
X_train = np.reshape(X_train_selected, (X_train_selected.shape[0], X_train_selected.shape[1], 1))
X_test = np.reshape(X_test_selected, (X_test_selected.shape[0], X_test_selected.shape[1], 1))


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

Define the LSTM model: Define the LSTM model with several LSTM layers and a dense layer for output.

In [ ]:
model = Sequential()

model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))


Compile the model: Compile the model with an appropriate optimizer and loss function

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')


model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))
